# Double UNet

The idea here is to train a new UNet that learns to convert the "raw" masks from the first UNet into the actual masks. I'll be using the model trained in the `t6_better_3dUNet` notebook.

In [2]:
import glob

import monai
from monai.networks.nets import UNet
from monai.transforms import (
    Compose,
    LoadNiftid,
    ScaleIntensityd,
    NormalizeIntensityd,
    AddChanneld,
    ToTensord,
    RandSpatialCropd,
    RandCropByPosNegLabeld,
    CropForegroundd,
    Identityd,
)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

import pytorch_lightning as pl
from sklearn.model_selection import train_test_split

In [3]:
class UNet_DF(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams
        
        self.unet = UNet(
            dimensions=3,
            in_channels=1,
            out_channels=2,
            channels=(64, 128, 258, 512, 1024),
            strides=(2, 2, 2, 2),
            norm=monai.networks.layers.Norm.BATCH,
            dropout=0,
        )
        self.sample_masks = []
    
    # Data setup
    def setup(self, stage):
        data_dir = 'data/'
        
        # Train imgs/masks
        train_imgs = []
        with open(data_dir + 'train_imgs.txt', 'r') as f:
            train_imgs = [image.rstrip() for image in f.readlines()]

        train_masks = []
        with open(data_dir + 'train_masks.txt', 'r') as f:
            train_masks = [mask.rstrip() for mask in f.readlines()]
        
        train_dicts = [{'image': image, 'mask': mask} for (image, mask) in zip(train_imgs, train_masks)]
        
        train_dicts, val_dicts = train_test_split(train_dicts, test_size=0.2)
        
        # Basic transforms
        data_keys = ["image", "mask"]
        data_transforms = Compose(
            [
                LoadNiftid(keys=data_keys),
                AddChanneld(keys=data_keys),
                NormalizeIntensityd(keys="image"),
                RandCropByPosNegLabeld(
                    keys=data_keys,
                    label_key="mask",
                    spatial_size=self.hparams.patch_size,
                    num_samples=4,
                    image_key="image"
                ),
            ]
        )
        
        self.train_dataset = monai.data.CacheDataset(
            data=train_dicts,
            transform=Compose(
                [
                    data_transforms,
                    ToTensord(keys=data_keys)
                ]
            ),
            cache_rate=1.0
        )
        
        self.val_dataset = monai.data.CacheDataset(
            data=val_dicts,
            transform=Compose(
                [
                    data_transforms,
                    ToTensord(keys=data_keys)
                ]
            ),
            cache_rate=1.0
        )
        
    def train_dataloader(self):
        return monai.data.DataLoader(
            self.train_dataset, batch_size=self.hparams.batch_size, shuffle=True, num_workers=hparams.num_workers
        )

    def val_dataloader(self):
        return monai.data.DataLoader(
            self.val_dataset, batch_size=self.hparams.batch_size, num_workers=hparams.num_workers
        )
    
    # Training setup
    def forward(self, image):
        return self.unet(image)
    
    def criterion(self, y_hat, y):
        dice_loss = monai.losses.DiceLoss(
            to_onehot_y=True,
            softmax=True
        )
        focal_loss = monai.losses.FocalLoss()
        return dice_loss(y_hat, y) + focal_loss(y_hat, y)
    
    def training_step(self, batch, batch_idx):
        inputs, labels = batch['image'], batch['mask']
        outputs = self(inputs)
        loss = self.criterion(outputs, labels)

        self.logger.log_metrics({"loss/train": loss}, self.global_step)

        return {'loss': loss}
    
    def configure_optimizers(self):
        lr = self.hparams.lr
        optimizer = torch.optim.Adam(self.unet.parameters(), lr=lr)
        return optimizer
    
    def validation_step(self, batch, batch_idx):
        inputs, labels = (
            batch["image"],
            batch["mask"],
        )
        outputs = self(inputs)
        
        # Sample masks
        if self.current_epoch != 0:
            image = outputs[0].argmax(0)[:, :, 8].unsqueeze(0).detach()
            self.sample_masks.append(image)
        
        loss = self.criterion(outputs, labels)
        return {"val_loss": loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        self.logger.log_metrics({"val/loss": avg_loss}, self.current_epoch)
        
        if self.current_epoch != 0:
            grid = torchvision.utils.make_grid(self.sample_masks)
            self.logger.experiment.add_image('sample_masks', grid, self.current_epoch)
            self.sample_masks = []
        
        return {"val_loss": avg_loss}

In [7]:
model_ckpts = glob.glob('models/7-8-2020_dicefocal/*.ckpt')
print(model_ckpts)
generator_model = UNet_DF.load_from_checkpoint(model_ckpts[0])

['models/7-8-2020_dicefocal/_ckpt_epoch_119.ckpt']


In [26]:
class MaskUNet(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams
        
        self.generator = hparams['generator']
        
        self.unet = UNet(
            dimensions=3,
            in_channels=1,
            out_channels=1,
            channels=(64, 128, 258, 512, 1024),
            strides=(2, 2, 2, 2),
            norm=monai.networks.layers.Norm.BATCH,
            dropout=0,
        )
        self.sample_masks = []
    
    # Data setup
    def setup(self, stage):
        data_dir = 'data/'
        
        # Train imgs/masks
        train_imgs = []
        with open(data_dir + 'train_imgs.txt', 'r') as f:
            train_imgs = [image.rstrip() for image in f.readlines()]

        train_masks = []
        with open(data_dir + 'train_masks.txt', 'r') as f:
            train_masks = [mask.rstrip() for mask in f.readlines()]
            
        train_dicts = [{'image': image, 'mask': mask} for (image, mask) in zip(train_imgs, train_masks)]
        
        train_dicts, val_dicts = train_test_split(train_dicts, test_size=0.2)
        
        # Basic transforms
        data_keys = ["image", "mask"]
        data_transforms = Compose(
            [
                LoadNiftid(keys=data_keys),
                AddChanneld(keys=data_keys),
                NormalizeIntensityd(keys="image"),
                RandCropByPosNegLabeld(
                    keys=data_keys,
                    label_key="mask",
                    spatial_size=self.hparams.patch_size,
                    num_samples=4,
                    image_key="image"
                ),
            ]
        )
        
        self.train_dataset = monai.data.CacheDataset(
            data=train_dicts,
            transform=Compose(
                [
                    data_transforms,
                    ToTensord(keys=data_keys)
                ]
            ),
            cache_rate=1.0
        )
        
        self.val_dataset = monai.data.CacheDataset(
            data=val_dicts,
            transform=Compose(
                [
                    data_transforms,
                    ToTensord(keys=data_keys)
                ]
            ),
            cache_rate=1.0
        )
    
    def train_dataloader(self):
        return monai.data.DataLoader(
            self.train_dataset, batch_size=self.hparams.batch_size, shuffle=True, num_workers=hparams.num_workers
        )

    def val_dataloader(self):
        return monai.data.DataLoader(
            self.val_dataset, batch_size=self.hparams.batch_size, num_workers=hparams.num_workers
        )
    def forward(self, image):
        return self.unet(image)
    
    def criterion(self, y_hat, y):
        dice_loss = monai.losses.DiceLoss(
            to_onehot_y=False,
            softmax=False
        )
        return dice_loss(y_hat, y)
    
    def training_step(self, batch, batch_idx):
        images, targets = batch['image'], batch['mask']
        inputs = self.generator(images).argmax(1).unsqueeze(1).type(torch.FloatTensor).cuda(images.device.index)
        outputs = self(inputs)
        loss = self.criterion(outputs, targets)

        self.logger.log_metrics({"loss/train": loss}, self.global_step)

        return {'loss': loss}
    
    def configure_optimizers(self):
        lr = self.hparams.lr
        optimizer = torch.optim.Adam(self.unet.parameters(), lr=lr)
        return optimizer
    
    def validation_step(self, batch, batch_idx):
        images, targets = batch['image'], batch['mask']
        inputs = self.generator(images).argmax(1).unsqueeze(1).type(torch.FloatTensor).cuda(images.device.index)
        outputs = self(inputs)
        
        # Sample masks
        if self.current_epoch != 0:
            image = torch.round(outputs[0][0][:, :, 8].unsqueeze(0).detach())
            self.sample_masks.append(image)
        
        loss = self.criterion(outputs, targets)
        return {"val_loss": loss}
    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        self.logger.log_metrics({"val/loss": avg_loss}, self.current_epoch)
        
        if self.current_epoch != 0:
            grid = torchvision.utils.make_grid(self.sample_masks)
            self.logger.experiment.add_image('sample_masks', grid, self.current_epoch)
            self.sample_masks = []
        
        return {"val_loss": avg_loss}

In [18]:
from argparse import Namespace

args = {
    'name': '7-9-2020_double_unet',
    'batch_size': 2,
    'lr': 0.001,
    'patch_size': [256, 256, 16],
    'num_workers': 6,
    'generator': generator_model,
}

hparams = Namespace(**args)

In [6]:
model = MaskUNet(hparams)

In [33]:
%reload_ext tensorboard
%load_ext tensorboard
%tensorboard --logdir models/7-9-2020_double_unet/tb_logs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 18780), started 20:02:32 ago. (Use '!kill 18780' to kill it.)

In [8]:
NAME = 'models/' + hparams.name
logger = pl.loggers.TensorBoardLogger(NAME + "/tb_logs/", name='')

# Callbacks
early_stopping = pl.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5
)

checkpoint_callback = pl.callbacks.ModelCheckpoint(filepath=NAME + '/checkpoints/')


trainer = pl.Trainer(
#     auto_scale_batch_size='binsearch',
    checkpoint_callback=checkpoint_callback,
    early_stop_callback=early_stopping,
    check_val_every_n_epoch=5,
    gpus=1,
    max_epochs=1000,
    logger=logger,
)

trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0,1]


115/115 Load and cache transformed data:  [==============================]
29/29 Load and cache transformed data:  [==============================]



  | Name      | Type    | Params
--------------------------------------
0 | generator | UNet_DF | 31 M  
1 | unet      | UNet    | 31 M  


1

## Evaluation

In [9]:
import os
import glob
import random
import matplotlib.pyplot as plt

In [30]:
model_ckpts = glob.glob('models/' + hparams.name + '/*.ckpt')
print(model_ckpts)
model = MaskUNet.load_from_checkpoint(
    model_ckpts[0],
    hparams=vars(hparams)
)

['models/7-9-2020_double_unet/_ckpt_epoch_4.ckpt']


KeyError: 'generator'

In [32]:
vars(hparams)

{'name': '7-9-2020_double_unet',
 'batch_size': 2,
 'lr': 0.001,
 'patch_size': [256, 256, 16],
 'num_workers': 6,
 'generator': UNet_DF(
   (unet): UNet(
     (model): Sequential(
       (0): Convolution(
         (conv): Conv3d(1, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
         (norm): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (act): PReLU(num_parameters=1)
       )
       (1): SkipConnection(
         (submodule): Sequential(
           (0): Convolution(
             (conv): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
             (norm): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
             (act): PReLU(num_parameters=1)
           )
           (1): SkipConnection(
             (submodule): Sequential(
               (0): Convolution(
                 (conv): Conv3d(128, 258, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
     